Perhaps mark each pdf with a code denoting the bank type, so this file only has to do a simple string parse to know what to do. i.e. 'JohnDoeCAP.pdf', 'MarkAnthonyNED.pdf', etc. 

Sometimes Description is long and overflows. I cut it off, but should we keep it all?

Formulae:
Number of transactions per month/amount of customers
Total money in/out subtract out-in for net
Bin description by types/prevalence 

In [1]:
import math 
import tabula as tb
from tabula import read_pdf

#constant conversion factor, per adobe
#pdf measurer pdf unit = 1/72 in
cf =72  

#all measurements in format [top,left,bottom,width]

#page counter
from PyPDF2 import PdfFileReader
def count_pdf_pages(file):
    with open(file, 'rb') as pdf:
        page_count = PdfFileReader(pdf).getNumPages()
    return page_count

In [ ]:
import pandas as pd
#NED Bank



#mercantile measurements
#first page:
page_1_box = [4.2, 0.4, 8.9, 7.7,]
#middle pages:
middle_pages_box = [2.4, 0.4, 10.1, 7.7]

page_count = count_pdf_pages(file)

#convert boxes
for i in range(0, len(page_1_box)):
    page_1_box[i] *= cf
    
for i in range(0, len(middle_pages_box)):
    middle_pages_box[i] *= cf

#get first page
page_1_table = tb.read_pdf(file, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
page_1_df = page_1_table[0].drop(labels='Post', axis=1)

#page_1_df
#clean first page
page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
page_1_df = page_1_df.drop(index=0)
######page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

#append first page
total_df = pd.DataFrame(columns=page_1_df.columns)
total_df = pd.concat([total_df, page_1_df])


for page in range(2, page_count+1):

    #get middle page
    middle_pages_table = tb.read_pdf(file, pages=page, area=[middle_pages_box],
                                 output_format="dataframe", stream=True, pandas_options={'header': None})
    middle_df = middle_pages_table[0]

    #clean middle page
    middle_df = middle_df.drop(labels=0, axis=1)
    middle_df = middle_df.drop(index=[0,1])
    middle_df = middle_df.dropna(axis='columns', how='all')
    middle_df.columns = page_1_df.columns
    middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

    #append to total df
    total_df = pd.concat([total_df, middle_df])





total_df


In [12]:
import pandas as pd
#Mercantile Bank
#can these be longer than the one provided??

    
def MER_scan(filename):

    #mercantile measurements
    #first page:
    page_1_box = [4.2, 0.4, 8.9, 7.7,]
    #middle pages:
    middle_pages_box = [2.4, 0.4, 10.1, 7.7]

    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Post', axis=1)

    #page_1_df
    #clean first page
    page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    ######page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.drop(labels=0, axis=1)
        middle_df = middle_df.drop(index=[0,1])
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns
        middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        return total_df


In [52]:
import pandas as pd
#CAPITEC
#complete

def CAP_scan(filename):
    #capitec measurements
    #first page:
    page_1_box = [3.77, 0.4, 9.8, 8.5,]
    #middle pages:
    middle_pages_box = [0.3, 0.4, 10.2, 8.5]

  
    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Posting Date', axis=1)

    #clean first page
    page_1_df = page_1_df.rename(columns={'Transaction': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns


        middle_df = middle_df.dropna(subset=['Transaction Date','Balance (R)'])
        middle_df['Transaction Date'] = middle_df['Transaction Date'].map(lambda x: str(x).split()[-1])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        
    #combine money in and money out columns
    total_df = total_df.bfill(axis=1)
    
    total_df = total_df.drop(['Money Out (R)'], axis=1)
    
    total_df = total_df.rename(columns={'Money In (R)': 'Amount'})
    return total_df




In [53]:
filename = "#CAP#Capitec.pdf"
df = CAP_scan(filename)
#elif filename.startswith("#FNB#"):
#    df = FNB_scan(filename)
#elif filename.startswith("#NED#"):
#    df = NED_scan(filename)
#elif filename.startswith("#MER#"):
#    df = MER_scan(filename)

df

,Transaction Date,Description,Amount,Balance (R)
1,30/08/2021,Branch Card Replacement Fee,-70.0,-57.64
2,30/08/2021,Cash Deposit Cash Dep Dnr Boksbur Dnr Boksbur...,100.0,42.36
4,30/08/2021,Cash Deposit Fee (Notes),-1.3,41.06
5,30/08/2021,Banking App Prepaid Purchase MTN,-15.0,26.06
6,30/08/2021,Prepaid Mobile Purchase Fee,-0.5,25.56
...,...,...,...,...
9,04/10/2021,ATM Cash Withdrawal Sparkatm Bp Roodekop,-100.0,338.9
11,04/10/2021,ATM Cash Withdrawal Fee,nk,331.4
12,04/10/2021,Payment Received: Rtc 05dtnty7sh Tk Capitec B2...,a,531.4
13,04/10/2021,Banking App Transfer to Rainy Days (1709362276...,-195.0,336.4


In [ ]:
filename = input("enter name of file to be scanned")
if filename.startswith("#CAP#"):
    df = CAP_scan(filename)
elif filename.startswith("#FNB#"):
    df = FNB_scan(filename)
elif filename.startswith("#NED#"):
    df = NED_scan(filename)
elif filename.startswith("#MER#"):
    df = MER_scan(filename)
print(df)


In [3]:
import pandas as pd

df1 = pd.DataFrame([['a', 1,2,3,4], ['b', 2],['a', 1], ['b', 2],['a', 1]],
                   columns=range(0,5))
df1

,0,1,2,3,4
0,a,1,2.0,3.0,4.0
1,b,2,NaN,NaN,NaN
2,a,1,NaN,NaN,NaN
3,b,2,NaN,NaN,NaN
4,a,1,NaN,NaN,NaN
